In [1]:
!pip install faiss-cpu
!pip install openai==0.28.0

In [2]:
# -*- coding: utf-8 -*-
"""app_for_colab.ipynb"""

import os
import faiss
import numpy as np
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import torch
import openai

# Load the CLIP model and processor
def load_clip_model():
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    return model, processor

clip_model, clip_processor = load_clip_model()

# Helper function to download files
def download_file(url, save_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(save_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return save_path
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to download {url}: {e}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Load FAISS indices
def load_faiss_indices():
    indices = {
        "text_only": "https://drive.google.com/uc?id=1c6GoqDAFgvVLzr4tcazLb6NS2ZbKRK_i",
        "image_only": "https://drive.google.com/uc?id=1CNfRQdtnokIckOjjkZgG4M_DZtV9_zJr",
        "multimodal_embeddings": "https://drive.google.com/uc?id=1lAKVwo1d0YK8KBtdk_A-UCzpVBH3Pl7d",
    }
    temp_dir = "temp_indices"
    os.makedirs(temp_dir, exist_ok=True)

    loaded_indices = {}
    for name, url in indices.items():
        save_path = os.path.join(temp_dir, f"{name}.index")
        if not os.path.exists(save_path):
            download_file(url, save_path)
        try:
            loaded_indices[name] = faiss.read_index(save_path)
        except Exception as e:
            print(f"Failed to load FAISS index for {name}: {e}")

    return (
        loaded_indices.get("text_only"),
        loaded_indices.get("image_only"),
        loaded_indices.get("multimodal_embeddings"),
    )

text_index, image_index, full_index = load_faiss_indices()

In [4]:
import pandas as pd
import gdown

# Step 1: Define the file path where the dataset will be saved
save_path = "final_dataset.csv"

# Step 2: Download the dataset using gdown
file_id = "104LiRhqzx4prsVQ__Lk-2rR92YzkDI7_"  # Replace with your file's ID
dataset_url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(dataset_url, save_path, quiet=False)

# Step 3: Load the dataset
try:
    final_dataset = pd.read_csv(save_path, encoding='utf-8')  # Adjust encoding if needed
    print("Dataset loaded successfully!")
    print(final_dataset.head())  # Display the first few rows of the dataset
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Downloading...
From (original): https://drive.google.com/uc?id=104LiRhqzx4prsVQ__Lk-2rR92YzkDI7_
From (redirected): https://drive.google.com/uc?id=104LiRhqzx4prsVQ__Lk-2rR92YzkDI7_&confirm=t&uuid=e99ad5ba-ae2d-47f3-b65d-f1fb39c7942c
To: /content/final_dataset.csv
100%|██████████| 31.8M/31.8M [00:00<00:00, 96.8MB/s]


Dataset loaded successfully!
                                Product Name_Cleaned  \
0  db longboards coreflex crossbow 41" bamboo fib...   
1  electronic snap circuits mini kits classpack, ...   
2  3doodler create flexy 3d printing filament ref...   
3  guillow airplane design studio with travel cas...   
4                   woodstock- collage 500 pc puzzle   

                                    Category_Cleaned  Selling Price_Cleaned  \
0  sports & outdoors   outdoor recreation   skate...                 237.68   
1  toys & games   learning & education   science ...                  99.95   
2          toys & games   arts & crafts   craft kits                  34.99   
3  toys & games   hobbies   models & model kits  ...                  28.91   
4            toys & games   puzzles   jigsaw puzzles                  17.49   

                               About Product_Cleaned  \
0  make sure this fits by entering your model num...   
1  make sure this fits by entering your model n

In [5]:
# Set OpenAI API key
openai.api_key = ""

In [6]:
# Helper functions
def preprocess_image(image):
    """Preprocess an image for CLIP."""
    return clip_processor(images=image, return_tensors="pt")

def generate_text_embeddings(texts):
    """Generate text embeddings using CLIP."""
    inputs = clip_processor(text=texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        return clip_model.get_text_features(**inputs).cpu().numpy()

def generate_image_embeddings(images):
    """Generate image embeddings using CLIP."""
    embeddings = []
    for image in images:
        inputs = preprocess_image(image)
        with torch.no_grad():
            embeddings.append(clip_model.get_image_features(**inputs).cpu().numpy())
    return np.vstack(embeddings)

In [7]:
def generate_response_gpt4(query, retrieved_items):
    """Generate a conversational response using GPT-4."""
    if retrieved_items.empty:
        return f"Sorry, I couldn't find any relevant products for your query: {query}"

    # Create a structured context from retrieved items
    context = "\n".join(
        [
            f"{i+1}. {row['Product Name_Cleaned']}: {row['About Product_Cleaned']} "
            f"(Category: {row['Category_Cleaned']}, Price: {row['Selling Price_Cleaned']}, "
            f"Image URL: {row['Image']})"
            for i, row in retrieved_items.iterrows()
        ]
    )

    # Create GPT-4 prompt
    prompt = f"""
    You are a helpful assistant recommending products based on user queries.

    Context:
    Here are some products relevant to the user's query:
    {context}

    Based on the query, recommend the most suitable product(s) and explain why they meet the user's needs.
    If the query asks for a specific product image, include the image URL in the response.

    Question: {query}
    Answer:
    """

    # Generate response using GPT-4
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a friendly, knowledgeable assistant helping users find products."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=300,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content']

In [10]:
# Usage in Colab
query_type = input("Select Query Type (Text, Image, Multimodal): ")

Select Query Type (Text, Image, Multimodal): Multimodal


In [11]:
query = input("Enter your query (for text and multimodal queries): ")

Enter your query (for text and multimodal queries): What is the name of this product, and how do I use it effectively?


In [15]:
from google.colab import files  # Import the files module

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the filename and save the uploaded file locally
uploaded_file_path = list(uploaded.keys())[0]
with open(uploaded_file_path, "wb") as f:
    f.write(uploaded[uploaded_file_path])  # Save the binary content to a file

Saving 1.jpg to 1.jpg


In [16]:
try:
    # Process text query
    text_embedding = None
    image_embedding = None

    if query_type in ["Text", "Multimodal"] and query:
        text_embedding = generate_text_embeddings([query])

    if query_type in ["Image", "Multimodal"] and uploaded_file_path:
        image = Image.open(uploaded_file_path).convert("RGB")
        image_embedding = generate_image_embeddings([image])

    # Combine embeddings for multimodal query
    if query_type == "Multimodal" and text_embedding is not None and image_embedding is not None:
        multimodal_embedding = np.hstack([text_embedding, image_embedding])
        index_to_use = full_index
    elif query_type == "Text" and text_embedding is not None:
        multimodal_embedding = text_embedding
        index_to_use = text_index
    elif query_type == "Image" and image_embedding is not None:
        multimodal_embedding = image_embedding
        index_to_use = image_index
    else:
        print("Please provide a valid query!")
        exit()

    # Perform search
    distances, indices = index_to_use.search(multimodal_embedding, k=1)

    # Ensure indices are within bounds
    valid_indices = [i for i in indices.flatten() if i < len(final_dataset)]
    retrieved_items = final_dataset.iloc[valid_indices]

    # Display results
    print("Retrieved Results:")
    for _, row in retrieved_items.iterrows():
        print(f"**{row['Product Name_Cleaned']}**")
        print(f"Category: {row['Category_Cleaned']}")
        print(f"Price: {row['Selling Price_Cleaned']}")
        print(f"[Product Link]({row['Product Url']})")
        print(f"Image: {row['Image']}")

    # Generate GPT-4 response
    gpt_response = generate_response_gpt4(query or "image-based query", retrieved_items)
    print("\nGPT-4 Response:")
    print(gpt_response)

except Exception as e:
    print(f"An error occurred: {e}")

Retrieved Results:
**pokemon tcg: sun and moon crimson invasion elite trainer box**
Category: nan
Price: 38.49
[Product Link](https://www.amazon.com/Pokemon-TCG-Crimson-Invasion-Trainer/dp/B074NCGNX2)
Image: https://images-na.ssl-images-amazon.com/images/I/51LFAQfMX6L.jpg|https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg

GPT-4 Response:
The name of the product is "Pokemon TCG: Sun and Moon Crimson Invasion Elite Trainer Box". 

This product is used for playing the Pokemon Trading Card Game. To use it effectively, you would first have to familiarize yourself with the rules of the game. The box includes a player's guide to the Sun & Moon—Crimson Invasion expansion, which should help you understand the specifics of this version of the game. 

The box includes 8 booster packs which contain random cards that you can add to your deck. The card sleeves featuring Silvally are used to protect your cards from damage. The 45 energy cards are a fundamental 

In [17]:
retrieved_items

,Product Name_Cleaned,Category_Cleaned,Selling Price_Cleaned,About Product_Cleaned,Product Specification_Cleaned,Technical Details_Cleaned,Shipping Weight_Cleaned,Image,Product Url,Is Amazon Seller_Cleaned,Combined_Text
9,pokemon tcg: sun and moon crimson invasion eli...,NaN,38.49,make sure this fits by entering your model num...,productdimensions:7.5x3.5x6.8inches itemweight...,show up to 2 reviews by default crimson choas ...,1.5,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Pokemon-TCG-Crimson-Inv...,True,pokemon tcg: sun and moon crimson invasion eli...
